In [59]:
import os
import re
import pandas as pd
from pybaseball import statcast_batter, statcast_pitcher, playerid_lookup, pitching_stats_range, batting_stats_range, schedule_and_record, team_game_logs, batting_stats
from datetime import timedelta, datetime
import statsapi
from statsapi import player_stat_data
import pprint

In [2]:
# Function to create a directory for player data if it doesn't exist
def create_player_dir(player_type):
    if not os.path.exists(player_type):
        os.makedirs(player_type)

In [133]:
# Function to store player data in a CSV file
def store_player_data(player_id, data, player_type):
    create_player_dir(player_type)
    file_path = f"{player_type}/{player_id}.csv"
    
    # Check if CSV file already exists for player_id
    if os.path.exists(file_path):
        existing_data = pd.read_csv(file_path)
        # Filter out rows that already exist in the existing_data
        data_to_add = data[~data.isin(existing_data)].dropna()
        if not data_to_add.empty:
            updated_data = pd.concat([existing_data, data_to_add])
            # Sort the data by the game date
            updated_data = updated_data.sort_values(by='game_date')
            updated_data.to_csv(file_path, index=False)
    else:
        # Sort the new data by the game date
        data = data.sort_values(by='game_date')
        data.to_csv(file_path, index=False)

In [119]:
from datetime import timedelta, datetime
import pandas as pd

# Function to fetch and store batter data
def fetch_and_store_batter_data(start_date, end_date, player_ids):
    for player_id in player_ids:
        try:
            # Initialize an empty list to collect data
            all_data = []

            # Loop over each date in the range and collect data
            current_date = start_date
            while current_date <= end_date:
                try:
                    fetched_data = batting_stats_range(current_date.strftime('%Y-%m-%d'))
                    player_data = fetched_data[fetched_data['mlbID'] == player_id]

                    if not player_data.empty:
                        all_data.append(player_data)
                except Exception as e:
                    print(f" Game-Data Error on {current_date.strftime('%Y-%m-%d')}: {e}")
                current_date += timedelta(days=1)

            # Concatenate all the collected data
            final_df = pd.concat(all_data)

            store_player_data(player_id, final_df, 'batters')
        except Exception as e:
            print(f"Error fetching data for batter {player_id}: {e}")

# Define the player IDs and the date range
player_ids = [677951]
end_date = datetime.now()- timedelta(days=15)
start_date = end_date - timedelta(days=1)  # end_date - timedelta(days=2*365)
start_date_str = start_date.strftime('%Y-%m-%d')
end_date_str = end_date.strftime('%Y-%m-%d')
#print(start_date_str, end_date_str)


fetch_and_store_batter_data(start_date, end_date, player_ids)


 Game-Data Error on 2024-05-08: list index out of range
 Game-Data Error on 2024-05-09: list index out of range
Error fetching data for batter 677951: No objects to concatenate


## Original Function Below

In [123]:
# Define the player's ID and the date range
player_id = 677951
end_date = datetime.today()
start_date = end_date - timedelta(days=10)         #end_date - timedelta(days=2*365)

# Function to get stats for a specific date and player
def get_player_stats(date, player_id):
    df = batting_stats_range(date.strftime('%Y-%m-%d'))
    return df[df['mlbID'] == player_id]

# Initialize an empty list to collect data
all_data = []

# Loop over each date in the range and collect data
current_date = start_date
while current_date <= end_date:
    try:
        player_data = get_player_stats(current_date, player_id)
        if not player_data.empty:
            all_data.append(player_data)
    except Exception as e:
        print(f"Error on {current_date.strftime('%Y-%m-%d')}: {e}")
    current_date += timedelta(days=1)

# Concatenate all the collected data
final_df = pd.concat(all_data)

# Display the resulting DataFrame
print(final_df)



Error on 2024-05-14: list index out of range
Error on 2024-05-15: list index out of range


KeyboardInterrupt: 

In [94]:
final_df.to_csv('witt.csv')

In [134]:
player_id = 677951
fetched_data = batting_stats_range('2024-05-23')
player_data = fetched_data[fetched_data['mlbID'] == player_id]

IndexError: list index out of range

In [125]:
def fetch_data_with_logging(date='2024-05-23'):
    try:
        data = batting_stats_range(date)
        if not data or 'mlbID' not in data.columns:
            raise ValueError("Incomplete data received")
        return data
    except Exception as e:
        print(f"Error fetching data for {date}: {e}")
        return pd.DataFrame()

In [129]:
print(fetch_data_with_logging())

Error fetching data for 2024-05-23: list index out of range
Empty DataFrame
Columns: []
Index: []
